In [1]:
import pandas as pd
import numpy as np
import shutil
import os
import imageio
from PIL import Image,ImageDraw,ImageFont

In [4]:
dataf='./SEU-ALLEN_Soma_Morphometry.csv'
df=pd.read_csv(dataf)
df.keys()

Index(['Soma ID', 'fMOST BrainID', 'Soma_X(Raw brain)', 'Soma_Y(Raw brain)',
       'Soma_Z(Raw brain)', 'Soma_X(CCFv3)', 'Soma_Y(CCFv3)', 'Soma_Z(CCFv3)',
       'CCF Region Registered', 'Local Traced', 'Full Traced',
       'Production Type (TeraVR or Hi5)'],
      dtype='object')

In [6]:
df['Full Traced']=0
df['Production Type (TeraVR or Hi5)']='Hi5'
tera_soma_df=pd.read_csv('./50k_soma.csv',index_col=['Brain'])
tera_labeled_brains=tera_soma_df.index.to_list()
df.loc[df['fMOST BrainID'].isin(tera_labeled_brains),'Production Type (TeraVR or Hi5)']='TeraVR'
df['Soma_X(CCFv3)']/=25
df['Soma_Y(CCFv3)']/=25
df['Soma_Z(CCFv3)']/=25

In [56]:
df

,Soma ID,fMOST BrainID,Soma_X(Raw brain),Soma_Y(Raw brain),Soma_Z(Raw brain),Soma_X(CCFv3),Soma_Y(CCFv3),Soma_Z(CCFv3),CCF Region Registered,Local Traced,Full Traced,Production Type (TeraVR or Hi5)
0,1,191797,9488.258,12907.284,1325.821,41.879,152.250,177.934,MOB,0,0,Hi5
1,2,191797,9520.883,12816.454,1345.435,42.905,152.916,176.329,MOB,0,0,Hi5
2,3,191797,9465.358,12822.361,1350.614,43.209,151.684,176.515,MOB,0,0,Hi5
3,4,191797,9446.882,12839.179,1498.692,50.940,151.525,177.062,MOB,0,0,Hi5
4,5,191797,9590.042,12979.944,1412.951,46.323,154.759,179.191,MOB,0,0,Hi5
...,...,...,...,...,...,...,...,...,...,...,...,...
227576,227577,196474,11048.715,12873.795,3538.461,176.984,215.920,201.528,ACB,0,0,Hi5
227577,227578,196474,9064.636,8877.950,3548.290,166.485,178.486,131.563,CP,1,0,Hi5
227578,227579,201604,16143.194,11152.736,2652.620,142.659,225.321,188.396,ACB,1,0,Hi5
227579,227580,201604,11178.039,10490.175,2600.956,146.551,224.365,263.771,ACB,1,0,Hi5


In [85]:
df.to_csv('./SEU-ALLEN_Morphometry.csv')

## full morphology

In [3]:
# get cell type
r1741_type='/Users/jiangshengdian/Desktop/Daily/Reconstructions/seu_allen_data/1741_Celltype.csv'
r1741df=pd.read_csv(r1741_type,index_col=['name'])
raw_feas_file='../../fullmorpho/py/ana_lm_statistics_full.csv'
# swcdir='../data/reg_ccf'
swcdir='../data/reg_ccf_s20'
fromFile=True
recompute_size=0
if fromFile and os.path.exists(raw_feas_file):
    all_features_raw=pd.read_csv(raw_feas_file,index_col=['name'])
else:
    for swc in allfeas.index:
        fbname=swc
        bid=fbname.split('_')[0]
        if bid == 'pre':
            bid=fbname.split('_')[1]
        if bid == '15257':
            fbname='210254_'+fbname.split('5257_')[1]
        if fbname in r1741df.index:
            soma_region=r1741df.loc[fbname,'Soma_region']
        else:
            swcf=os.path.join(swcdir,swc+'.swc')
            soma_region=get_soma_region(swcf)
            
        allfeas.loc[swc,'soma_region']=soma_region

In [5]:
allsomadf=all_features_raw['soma_region'].copy()
allsoma='/Users/jiangshengdian/Desktop/Daily/PhD_project/Platform/SD/data/1886_Soma_region.csv'
allsomadf.to_csv(allsoma)

In [6]:
def readswc(file,out_fea=None,fea_val_names=None,header=None,skiprows=3):
    swc=pd.read_csv(file,sep=' ',header=None,skiprows=skiprows)
    if header is None:
        header=['n','type','x','y','z','radius','parent']
        if swc.shape[1] >= 12:
            header+=['seg_id','level','mode','timestamp','teraflyindex']
            for i in np.arange(12,swc.shape[1]):
                if fea_val_names is not None:
                    header.append(fea_val_names[i-12])
                else:
                    header.append('fea_'+str(i-12))
    if len(header) == swc.shape[1]:
        swc.columns=header
    swc.set_index(['n'],drop=True,inplace=True)
    if out_fea is None:
        return swc
    else:
        return swc[out_fea]
def get_rid(swc):
    '''
    Find root node.
    '''
    rnode=swc[((swc['parent']<0) & (swc['type']<=1))]
    if rnode.shape[0]<1:
        return -1
    return rnode.index[0]
# get cell type
r1741_type='/Users/jiangshengdian/Desktop/Daily/Reconstructions/seu_allen_data/1741_Celltype.csv'
r1741df=pd.read_csv(r1741_type,index_col=['name'])
allsoma='/Users/jiangshengdian/Desktop/Daily/PhD_project/Platform/SD/data/1886_Soma_region.csv'
allsomadf=pd.read_csv(allsoma,index_col=['name'])
fdir='/Users/jiangshengdian/Desktop/Daily/PhD_project/Platform/SD/release/full_morpho/RAW'
fsomadf=pd.DataFrame()
for root,dirs,files in os.walk(fdir):
    for mfile in files:
        if mfile == '.DS_Store' or mfile[0] == '.':
            continue
        file_in=os.path.join(root,mfile)
        fname=mfile.split('.swc')[0]
        bid=fname.split('_')[0]
        if bid == 'pre':
            bid=fname.split('_')[1]
        swc=readswc(file=file_in)
        rid=get_rid(swc=swc)
        if rid < 0:
            print('wrong file: ',mfile)
            continue
        fsomadf.loc[fname,'fMOST Brain ID']=bid
        fsomadf.loc[fname,'Soma_X(Raw brain, in voxel)']=swc.loc[rid,'x']
        fsomadf.loc[fname,'Soma_Y(Raw brain, in voxel)']=swc.loc[rid,'y']
        fsomadf.loc[fname,'Soma_Z(Raw brain, in voxel)']=swc.loc[rid,'z']
rfdir='/Users/jiangshengdian/Desktop/Daily/PhD_project/Platform/SD/release/full_morpho/CCFv3'
for root,dirs,files in os.walk(rfdir):
    for mfile in files:
        if mfile == '.DS_Store' or mfile[0] == '.':
            continue
        file_in=os.path.join(root,mfile)
        fname=mfile.split('.swc')[0]
        swc=readswc(file=file_in)
        rid=get_rid(swc=swc)
        if rid < 0:
            print('wrong file: ',mfile)
            continue
        fsomadf.loc[fname,'Soma_X(CCFv3)']=swc.loc[rid,'x']
        fsomadf.loc[fname,'Soma_Y(CCFv3)']=swc.loc[rid,'y']
        fsomadf.loc[fname,'Soma_Z(CCFv3)']=swc.loc[rid,'z']
fsomadf.index.name='Name'

In [12]:
fsomadf['Manually annotated CCF region of soma']=0
fsomadf['Projection class']='NA'
fsomadf['Cortical Lamination of soma']='n/a'
for cell in fsomadf.index:
    fsomadf.loc[cell,'Registered CCF Region of soma']=allsomadf.loc[cell,'soma_region']
    
    if cell in r1741df.index:
        fsomadf.loc[cell,'Manually annotated CCF region of soma']=1
        if r1741df.loc[cell,'Subclass_or_type'] is not None:
            fsomadf.loc[cell,'Projection class']=r1741df.loc[cell,'Subclass_or_type']
    # if cell in r1741df.index:
        cortical_layer=r1741df.loc[cell,'Cortical_layer']
    fsomadf.loc[cell,'Cortical Lamination of soma']=str(cortical_layer)
fsomadf.to_csv('../release/Full_morphometry.csv')

In [13]:
fsomadf.to_csv('../release/Full_morphometry.csv')

In [14]:
fsomadf.to_excel('../release/Full_morphometry.xlsx')

In [82]:
df['Full Traced']=0
fsomadf['in_227k']=0
for soma in fsomadf.index:
    bname=soma.split('.swc')[0]
    bid=bname.split('_')[0]
    if bid == 'pre':
        bid=bname.split('_')[1]
    rx=fsomadf.loc[soma,'x']
    ry=fsomadf.loc[soma,'y']
    rz=fsomadf.loc[soma,'z']
    # matchedf=df[(df['fMOST BrainID']==np.int32(bid))&
    #                     (abs(df['Soma_X(Raw brain)']-rx)==0)&
    #                     (abs(df['Soma_Y(Raw brain)']-ry)==0)&
    #                     (abs(df['Soma_Z(Raw brain)']-rz)==0)].copy()
    inbdf=df[df['fMOST BrainID']==np.int32(bid)].copy()
    # min_dist2_227k=
    if matchedf.shape[0] == 1:
        fsomadf.loc[soma,'in_227k']=matchedf.shape[0]
        df.loc[matchedf.index[0],'Full Traced']=1
        # pass

Data Comparison of soma_region

In [24]:
old_soma_region='/Users/jiangshengdian/Desktop/Daily/PhD_project/Platform/ms/get_brain_meta/1891_Soma_region.csv'
somdf_1891=pd.read_csv(old_soma_region,index_col=['name'])
allsoma='/Users/jiangshengdian/Desktop/Daily/PhD_project/Platform/SD/data/1886_Soma_region.csv'
somdf_1886=pd.read_csv(allsoma,index_col=['name'])
for cell in somdf_1886.index:
    new_region=somdf_1886.loc[cell,'soma_region']
    old_region=somdf_1891.loc[cell,'soma_region']
    if new_region != old_region:
        print(cell,old_region,new_region)

In [26]:
print(somdf_1886.loc['182724_3887_x5901_y10339'])
print(somdf_1891.loc['182724_3887_x5901_y10339'])

soma_region    SSp-ul
Name: 182724_3887_x5901_y10339, dtype: object
soma_region    SSp-ul
Name: 182724_3887_x5901_y10339, dtype: object


In [28]:
r1741df.loc['182724_3887_x5901_y10339','Soma_region']

Soma_region                           SSp-ul
Cortical_layer                           NaN
Transgenic_line     Fezf2-CreER;Ai166-405426
Brain_id                              182724
Subclass_or_type                      CTX_ET
Celltype                           ET_SSp-ul
Name: 182724_3887_x5901_y10339, dtype: object

Data cmp of stable

In [38]:
smy='/Users/jiangshengdian/Desktop/Daily/PhD_project/Platform/SD/release/Full_morphometry_bk.csv'
syf='/Users/jiangshengdian/Desktop/Daily/PhD_project/Platform/SD/release/TableS6_Full_morphometry_1213.csv'
sdf_my=pd.read_csv(smy,index_col=['Name'])
sdf_yf=pd.read_csv(syf,index_col=['Name'])

In [57]:
# cmp_keys=sdf_my.keys()
cmp_keys=['fMOSTID','Soma_X', 'Soma_Y', 'Soma_Z', 'Soma_X(CCFv3)','Soma_Y(CCFv3)', 'Soma_Z(CCFv3)', 'Manuallysoma', 'Registeredsoma']
for cell in sdf_my.index:
    wrong_match=False
    for key in cmp_keys:
        myv=sdf_my.loc[cell,key]
        yfv=sdf_yf.loc[cell,key]
        if myv is None and yfv is None:
            continue
        if myv != yfv:
            wrong_match=True
            break
    if wrong_match and (myv != 'error'):
        if key == 'Manuallysoma':
            continue
        if myv == 'fiber tracts':
            continue
        print(cell,key,myv,yfv)

pre_18464_00102 Registeredsoma CA1 DG
18463_00014 Registeredsoma VPM nan
18458_00492 Registeredsoma VPL VPM


In [50]:
cell='pre_18464_00102'
print(somdf_1886.loc[cell])
print(somdf_1891.loc[cell])

soma_region    CA1
Name: pre_18464_00102, dtype: object
soma_region    CA1
Name: pre_18464_00102, dtype: object


## local morphology
rename to somaid

In [2]:
somaf='/Users/jiangshengdian/Desktop/Daily/PhD_project/Platform/webpage/data/TableS3_Soma_morphometry_1222.csv'
somadf=pd.read_csv(somaf)
soma_d15k=somadf[somadf['SEU-D15K']==1].copy()

In [9]:
soma_d15k.shape

(15441, 13)

In [4]:
somadf.keys()

Index(['Soma ID', 'fMOST BrainID', 'Soma_X(Raw brain, in voxel)',
       'Soma_Y(Raw brain, in voxel)', 'Soma_Z(Raw brain, in voxel)',
       'Soma_X(CCFv3_1𝜇𝑚)', 'Soma_Y(CCFv3_1𝜇𝑚)', 'Soma_Z(CCFv3_1𝜇𝑚)',
       'Production platform', 'Registered  CCF Region', 'Local Traced',
       'SEU-D15K', 'isManuallyChecked'],
      dtype='object')

In [8]:
# match of soma: D15K raw
swcdir='/Users/jiangshengdian/Desktop/Daily/PhD_project/Platform/SD/release/local_morphology/20230510/RAW_coord_in_brain'
todir='/Users/jiangshengdian/Desktop/Daily/PhD_project/Platform/SD/release/local_morphology/D15K_RAW'
if not os.path.exists(todir):
    os.mkdir(todir)
outf=0
not_matched_files=0
for root,dirs,files in os.walk(swcdir):
    for brain in dirs:
        bdir=os.path.join(root,brain)
        tobdir=os.path.join(todir,brain)
        if not os.path.exists(tobdir):
            os.mkdir(tobdir)
        for root1,dirs1,swcs in os.walk(bdir):
            for mfile in swcs:
                if mfile == '.DS_Store' or mfile[0] == '.':
                    continue
                
                file_in=os.path.join(root1,mfile)
                fname=mfile.split('.v3d')[0]
                rx=2*np.float64(fname.split('_')[2])
                ry=2*np.float64(fname.split('_')[4])
                rz=2*np.float64(fname.split('_')[6])
                matchedf=soma_d15k[(soma_d15k['fMOST BrainID']==np.int32(brain))&
                            (abs(soma_d15k['Soma_X(Raw brain, in voxel)']-rx)<1)&
                            (abs(soma_d15k['Soma_Y(Raw brain, in voxel)']-ry)<1)&
                            (abs(soma_d15k['Soma_Z(Raw brain, in voxel)']-rz)<1)].copy()
                if matchedf.shape[0] != 1:
                    # print(brain,fname)
                    # not_matched_files+=1
                    continue
                msoma_index=matchedf.index[0]
                outname=str(soma_d15k.loc[msoma_index,'Soma ID'])+'.swc'
                tof=os.path.join(tobdir,outname)
                if not os.path.exists(tof):
                    outf+=1
                    shutil.copyfile(file_in,tof)
                # soma_d15k.loc[msoma_index,'Soma ID']=1
                # soma_d15k.loc[msoma_index,'swc_name']=fname
                # outf+=1
print('processing files: '+str(outf))
# print('left files: '+ str(not_matched_files))

processing files: 15441


In [10]:
# match of soma: D15K CCFv3
# Img_X_4118.08_Y_21034_Z_1959.07.v3dpbd.swc
# Img_X_2938.56_Y_2634.03_Z_2412.32.v3dpbd_stps.swc
swcdir='/Users/jiangshengdian/Desktop/Daily/PhD_project/Platform/SD/release/local_morphology/20230510/RAW_coord_in_brain'
regswcdir='/Users/jiangshengdian/Desktop/Daily/PhD_project/Platform/SD/release/local_morphology/20230510/CCFv3_25um'
todir='/Users/jiangshengdian/Desktop/Daily/PhD_project/Platform/SD/release/local_morphology/D15K_CCFv3_25um'
if not os.path.exists(todir):
    os.mkdir(todir)
outf=0
not_matched_files=0
for root,dirs,files in os.walk(swcdir):
    for brain in dirs:
        bdir=os.path.join(root,brain)
        tobdir=os.path.join(todir,brain)
        if not os.path.exists(tobdir):
            os.mkdir(tobdir)
        for root1,dirs1,swcs in os.walk(bdir):
            for mfile in swcs:
                if mfile == '.DS_Store' or mfile[0] == '.':
                    continue
                file_in=os.path.join(regswcdir,brain,mfile.replace('pbd.swc','pbd_stps.swc'))
                fname=mfile.split('.v3d')[0]
                rx=2*np.float64(fname.split('_')[2])
                ry=2*np.float64(fname.split('_')[4])
                rz=2*np.float64(fname.split('_')[6])
                matchedf=soma_d15k[(soma_d15k['fMOST BrainID']==np.int32(brain))&
                            (abs(soma_d15k['Soma_X(Raw brain, in voxel)']-rx)<1)&
                            (abs(soma_d15k['Soma_Y(Raw brain, in voxel)']-ry)<1)&
                            (abs(soma_d15k['Soma_Z(Raw brain, in voxel)']-rz)<1)].copy()
                if matchedf.shape[0] != 1:
                    # print(brain,fname)
                    # not_matched_files+=1
                    continue
                msoma_index=matchedf.index[0]
                outname=str(soma_d15k.loc[msoma_index,'Soma ID'])+'.swc'
                tof=os.path.join(tobdir,outname)
                if not os.path.exists(tof) and os.path.exists(file_in):
                    outf+=1
                    shutil.copyfile(file_in,tof)
                # soma_d15k.loc[msoma_index,'Soma ID']=1
                # soma_d15k.loc[msoma_index,'swc_name']=fname
                # outf+=1
print('processing files: '+str(outf))
# print('left files: '+ str(not_matched_files))

processing files: 15441


## For Web data

In [148]:
'''For soma morphometry
1. read soma morphometry
2. convert to marker
3. render in vtk and generate image
4. image seq to video
'''
import random
somaf='/Users/jiangshengdian/Desktop/Daily/PhD_project/Platform/webpage/data/TableS3_Soma_morphometry_1222.csv'
somadf=pd.read_csv(somaf)
soma_in_brain=somadf[['Soma ID', 'fMOST BrainID', 'Soma_X(CCFv3_1𝜇𝑚)', 'Soma_Y(CCFv3_1𝜇𝑚)', 'Soma_Z(CCFv3_1𝜇𝑚)','Registered  CCF Region']].copy()
head='##n,orderinfo,name,comment,z,x,y,pixmax,intensity,sdev,volsize,mass,f1,f2,f3,color_r,color_g,color_b'
#0,,18454,,3964.11,8454.241,4969.068,0,0,0,100,0,,,,125.0,250.0,196.0
for bid in somadf['fMOST BrainID'].value_counts().index.to_list():
    somas=soma_in_brain[soma_in_brain['fMOST BrainID']==bid].copy()
    region_sorted=somas['Registered  CCF Region'].value_counts().index.to_list()
    somas['r']=0
    somas['g']=0
    somas['b']=0
    somas['c']=-1
    for i,region in enumerate(region_sorted):
       somas.loc[somas['Registered  CCF Region']==region,'c']=i
       clist=random.choices(np.arange(255),k=3)
       somas.loc[somas['Registered  CCF Region']==region,'r']=clist[0]
       somas.loc[somas['Registered  CCF Region']==region,'g']=clist[1]
       somas.loc[somas['Registered  CCF Region']==region,'b']=clist[2]
    soma_markers=pd.DataFrame()
    for cell in somas.index:
       if somas.loc[cell,'c'] < 0:
          continue
       soma_markers.loc[cell,'##x']=int(somas.loc[cell,'Soma_X(CCFv3_1𝜇𝑚)']/25+20)
       soma_markers.loc[cell,'y']=int(somas.loc[cell,'Soma_Y(CCFv3_1𝜇𝑚)']/25)
       soma_markers.loc[cell,'z']=int(somas.loc[cell,'Soma_Z(CCFv3_1𝜇𝑚)']/25)
       soma_markers.loc[cell,'radius']=0
       soma_markers.loc[cell,'shape']=1
       soma_markers.loc[cell,'name']=somas.loc[cell,'Registered  CCF Region']
       soma_markers.loc[cell,'comment']=somas.loc[cell,'c']
       soma_markers.loc[cell,'color_r']=somas.loc[cell,'r']
       soma_markers.loc[cell,'color_g']=somas.loc[cell,'g']
       soma_markers.loc[cell,'color_b']=somas.loc[cell,'b']
    soma_markers.to_csv('/Users/jiangshengdian/Desktop/Daily/PhD_project/Platform/webpage/data/soma_markers/'+str(bid)+'.marker',index=False,float_format='%.0f')

In [5]:
# image sequence to video
image_src='/Users/jiangshengdian/Desktop/Daily/PhD_project/Platform/webpage/data/soma_figs'
# 读取所有 PNG 图片
color=(255,0,0)
images = []
for file_name in sorted(os.listdir(image_src)):
    if file_name.endswith('.png'):
        fpath=os.path.join(image_src,file_name)
        fimage=Image.open(fpath).resize([600,600]).rotate(270)
        drawimage=ImageDraw.Draw(fimage)
        text="Brain : "+file_name.split('.')[0]
        font=ImageFont.truetype('Arial Unicode.ttf', size=50)
        # text_width, text_height = drawimage.textsize(text, font)
        x = 150
        y = 30
        drawimage.text((x, y), text, font=font, fill=color)
        # break
        images.append(fimage)
images=np.array(images)
# fimage
# 将图片转换为视频
fps = 1  # 每秒钟30帧
filepath='/Users/jiangshengdian/Desktop/Daily/PhD_project/Platform/webpage/data/soma.mp4'
with imageio.get_writer(filepath, fps=fps) as video:
    for image in images:
        frame = image
        video.append_data(frame)

IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (600, 600) to (608, 608) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).
[swscaler @ 0x7f8f3f557000] Warning: data is not aligned! This can lead to a speed loss


In [6]:
'''For local morphometry:
1. morphology by image id
2. vtk
3. to video
'''

image_src='/Users/jiangshengdian/Desktop/Daily/PhD_project/Platform/webpage/data/local_figs'
# 读取所有 PNG 图片
color=(255,0,0)
images = []
for file_name in sorted(os.listdir(image_src)):
    if file_name.endswith('.png'):
        fpath=os.path.join(image_src,file_name)
        fimage=Image.open(fpath).resize([600,600]).rotate(270)
        drawimage=ImageDraw.Draw(fimage)
        text="Brain : "+file_name.split('.')[0]
        font=ImageFont.truetype('Arial Unicode.ttf', size=50)
        # text_width, text_height = drawimage.textsize(text, font)
        x = 150
        y = 30
        drawimage.text((x, y), text, font=font, fill=color)
        # break
        images.append(fimage)
images=np.array(images)
# fimage
# 将图片转换为视频
fps = 1  # 每秒钟30帧
filepath='/Users/jiangshengdian/Desktop/Daily/PhD_project/Platform/webpage/data/local.mp4'
with imageio.get_writer(filepath, fps=fps) as video:
    for image in images:
        frame = image
        video.append_data(frame)

IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (600, 600) to (608, 608) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).
[swscaler @ 0x7fc9f5257000] Warning: data is not aligned! This can lead to a speed loss


In [7]:
'''For full morphometry:
1. morphology by image id
2. vtk
3. to video
'''

image_src='/Users/jiangshengdian/Desktop/Daily/PhD_project/Platform/webpage/data/full_figs'
# 读取所有 PNG 图片
color=(255,0,0)
images = []
for file_name in sorted(os.listdir(image_src)):
    if file_name.endswith('.png'):
        fpath=os.path.join(image_src,file_name)
        fimage=Image.open(fpath).resize([600,600]).rotate(270)
        drawimage=ImageDraw.Draw(fimage)
        text="Brain : "+file_name.split('.')[0]
        font=ImageFont.truetype('Arial Unicode.ttf', size=50)
        # text_width, text_height = drawimage.textsize(text, font)
        x = 150
        y = 30
        drawimage.text((x, y), text, font=font, fill=color)
        # break
        images.append(fimage)
images=np.array(images)
# fimage
# 将图片转换为视频
fps = 1  # 每秒钟30帧
filepath='/Users/jiangshengdian/Desktop/Daily/PhD_project/Platform/webpage/data/full.mp4'
with imageio.get_writer(filepath, fps=fps) as video:
    for image in images:
        frame = image
        video.append_data(frame)

IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (600, 600) to (608, 608) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).
[swscaler @ 0x7f9ef0a57000] Warning: data is not aligned! This can lead to a speed loss


In [2]:
'''For bouton morphometry:
'''
all1876_feas_file='../../fullmorpho/py/ana_lm_statistics_full.csv'
all_1876_feas=pd.read_csv(all1876_feas_file,index_col=['name'])
image_src='/Users/jiangshengdian/Desktop/Daily/PhD_project/Platform/webpage/data/bouton_figs'
# 读取所有 PNG 图片
color=(255,0,0)
images = []
for region in all_1876_feas['soma_region'].value_counts().keys().tolist():
    tregion=all_1876_feas[all_1876_feas['soma_region']==region].copy()
    for cell in tregion.index:
        file_name=cell+'.png'
        fpath=os.path.join(image_src,file_name)
        if os.path.exists(fpath):
            fimage=Image.open(fpath).resize([600,600])
            drawimage=ImageDraw.Draw(fimage)
            text=region
            font=ImageFont.truetype('Arial Unicode.ttf', size=38)
            # text_width, text_height = drawimage.textsize(text, font)
            x = 10
            y = 50
            drawimage.text((x, y), text, font=font, fill=color)
            # break
            images.append(fimage)

images=np.array(images)
# fimage
# # 将图片转换为视频
fps = 3  # 每秒钟30帧
filepath='/Users/jiangshengdian/Desktop/Daily/PhD_project/Platform/webpage/data/bouton_mst.mp4'
with imageio.get_writer(filepath, fps=fps) as video:
    for image in images:
        frame = image
        video.append_data(frame)

IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (600, 600) to (608, 608) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).


In [36]:
fps = 2  # 每秒钟30帧
filepath='/Users/jiangshengdian/Desktop/Daily/PhD_project/Platform/webpage/data/bouton_mst.mp4'
with imageio.get_writer(filepath, fps=fps) as video:
    for image in images:
        frame = image
        video.append_data(frame)

IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (600, 600) to (608, 608) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).


In [147]:
'''Resize Fig data'''
main_figs='/Users/jiangshengdian/Desktop/Daily/PhD_project/Platform/webpage/data/main_figs'
for i in np.arange(1,9):
    figpath=os.path.join(main_figs,'Fig'+str(i)+'.png')
    fimage=Image.open(figpath).resize([1200,1000])
    tofigpath=os.path.join(main_figs,'Fig'+str(i)+'_resized.png')
    fimage.save(tofigpath)
    # break